<a href="https://colab.research.google.com/github/F423058/socu-ids-prg01-2024-ex01--F423058-/blob/main/ex10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第10回課題

In [1]:
# このセルは決して書き換えたり消したりしないこと
import os

def local_run(t):
    """
    自動採点を妨げずにテストを実行するための補助関数。
    """
    if not ('GRADING' in os.environ):
        t()

## Q1. 成績評価プログラム

- 各学生の中間試験の点数を記録した辞書`mid`と、
- 各学生の期末試験の点数を記録した辞書`fin`

が与えられた時、各学生の評価を記録した辞書を作成する関数`grading()`を定義せよ。点数を記録した辞書において、キーと値は以下のようになっているものとする。
- キー: 学籍番号を表す文字列
- 値: 点数を表す整数(0以上100以下)

成績は以下で定義する合計点数:

$$合計点数 = 中間試験の点数\times 0.4 + 期末試験の点数\times 0.6$$

に基づいて、以下の表の通りにつけるものとする。

| 成績 | 合計点数 |
|------|--------|
| 'S' | 90以上100以下 |
| 'A' | 80以上90未満 |
| 'B' | 70以上80未満 |
| 'C' | 60以上70未満 |
| 'D' | 60未満 |

ただし、
- `mid`には点数が記録されているが`fin`には記録されていない学生の期末試験の点数は0点
- `fin`には点数が記録されているが`mid`には記録されていない学生の中間試験の点数は0点

として計算せよ。

なお、この成績評価方法は架空のものであり、実在の授業における成績評価とは一切関係ない。

### Step 1. 具体例を考え、テストを定義

In [3]:
def test_grading():
    """
    grading()をテストする。
    """
    mid = {'a150001' : 35}
    fin = {'a150002' : 100}
    assert grading(mid, fin) == {'a150001' : 'D', 'a150002' : 'C'}
    # 以下必要に応じてテストケースを追加せよ(間違わない自信があれば追加しなくてもよい)
    mid = {'a150005': 0}
    fin = {'a150005': 0}
    assert grading(mid, fin) == {'a150005': 'D'}

### Step 2. `grading()`を定義

In [2]:
def grading(mid, fin):
    """
    中間試験と期末試験の点数から成績をつける

    Parameters:
        mid (dict): 中間試験の点数を記録した辞書
        fin (dict): 期末試験の点数を記録した辞書

    Returns:
        dict: 成績を記録した辞書
    """
    students = mid.keys() | fin.keys()
    grades = {}

    for student in students:
        mid_score = mid.get(student, 0)
        fin_score = fin.get(student, 0)

        total_score = mid_score * 0.4 + fin_score * 0.6

        if total_score >= 90:
            grade = 'S'
        elif total_score >= 80:
            grade = 'A'
        elif total_score >= 70:
            grade = 'B'
        elif total_score >= 60:
            grade = 'C'
        else:
            grade = 'D'

        grades[student] = grade

    return grades

### Step 3. テストを実行

In [4]:
# このセルは決して書き換えたり消したりしないこと
local_run(test_grading)

## Q2. お絵かきロジックの作問プログラム

### お絵かきロジック

> お絵かきロジック（おえかきロジック）は、縦と横の数字をヒントに塗り潰すマス目を割り出し、そのとおりに塗り潰していくと、最終的に絵（または文字）が浮かび上がるタイプのペンシルパズル。ののぐらむ（ノノグラム）、イラストロジック（イラロジ）、ピクロスとも呼ばれている。

![](https://upload.wikimedia.org/wikipedia/commons/6/64/Paint_by_numbers_Animation.gif)

- Wikipedia: https://ja.wikipedia.org/wiki/%E3%81%8A%E7%B5%B5%E3%81%8B%E3%81%8D%E3%83%AD%E3%82%B8%E3%83%83%E3%82%AF
- MiniCGI「お絵かきロジック」で遊ぶ: https://www.minicgi.net/logic/

### 問題

あなたはお絵かきロジックの問題作成の手伝いを頼まれた。渡された正解の絵ができあがるように、問題(上および左に表示される数字の列)を作成してほしいというのである。作成した問題の数だけ、謝金が支払われるらしい。

手作業で黒のマスを数えていってもよいが、それでは一問あたりに膨大な時間がかかる。コンピュータに問題の作成をやらせることができれば、あなたはコンピュータに正解の絵を与えるだけでよくなり、1時間あたり100問作ることも可能になるはずだ(正解の絵を作るのはあなたの仕事ではない)。

正解の絵は塗りつぶされたマス'■'と塗りつぶされていないマス'□'を要素とする文字列のリストで表現することにしよう。例えば花の絵(チューリップか、某配管工兄弟が火の玉を投げられるようになる花かは知らない)は以下のように表される。

```python
["□□□■□□□■□□□■□□□",
 "□□□■■□■■■□■■□□□",
 "□□□■■■■■■■■■□□□",
 "□□□■■■■■■■■■□□□",
 "□□□■■■■■■■■■□□□",
 "□□□■■■■■■■■■□□□",
 "□□□□■■■■■■■□□□□",
 "□□□□□□■■■□□□□□□",
 "□□□□□□□■□□□□□□□",
 "■■■■□□□■□□□■■■■",
 "■■■■□□□■□□□■■■■",
 "■■■■■■□■□■■■■■■",
 "■■■■■■□■□■■■■■■",
 "□□■■■■□■□■■■■□□",
 "□□■■■■■■■■■■■□□",
 "□□□□□■■■■■□□□□□"]
```


便利のため、以下の通り、塗りつぶされているマスを表す定数`FULL`と塗りつぶされていないマスを表す定数`EMPTY`を定義する。

In [8]:
FULL = '■'  # 塗りつぶされているマス
EMPTY = '□' # 塗りつぶされていないマス

#### Q2-1

絵の一行を表す文字列が与えられた時、行(横方向)の塗りつぶしを指示する数字の列(整数のリスト)を返す関数`get_sequence_lengths()`を定義せよ。結果のリストは先頭の要素ほど左の数値を表すものとする。例えば`[2,3]`は2マス塗りつぶしが続き、その右に1マス以上塗りつぶさないマスがあった後、3マス塗りつぶしが続くことを表すものとする。

##### Step 1. 具体例を考え、テストを定義

In [13]:
def test_get_sequence_lengths():
    """
    get_sequence_lengths()をテストする。
    """
    assert get_sequence_lengths("□□□■□□□■□□□■□□□") == [1,1,1]
    assert get_sequence_lengths("□□□■■□■■■□■■□□□") == [2,3,2]
    # 以下必要に応じてテストケースを追加せよ(間違わない自信があれば追加しなくてもよい)
    assert get_sequence_lengths("□□□■□□□■□□□■□□□") == [1, 1, 1]

##### Step 2. `get_sequence_lengths()`を定義

In [10]:
def get_sequence_lengths(r):
    """
    横方向の塗りつぶしを指示する数字の列(リスト)を返す

    Parameters:
        r (str): 絵の一行を表す文字列

    Returns:
        list: 塗りつぶしを指示する数字のリスト
    """
    lengths = []
    current_length = 0

    for cell in r:
        if cell == FULL:
            current_length += 1
        elif cell == EMPTY:
            if current_length > 0:
                lengths.append(current_length)
                current_length = 0

    if current_length > 0:
        lengths.append(current_length)

    return lengths

##### Step 3. テストの実行

In [14]:
# このセルは決して書き換えたり消したりしないこと
local_run(test_get_sequence_lengths)

#### Q2-2

絵を表す文字列のリストを転置して、行を表す文字列のリストから、列を表す文字列のリストを作る関数を作る関数`transpose()`を定義せよ。絵$P$

$$P = \left[\begin{array}{c}
"p_{(1,1)} \cdots p_{(1,n)}",\\
\vdots \\
"p_{(m,1)} \cdots p_{(m,n)}"\\
\end{array}\right]$$

の転置$P^T$は以下のようになる。

$$P^T = \left[\begin{array}{c}
"p_{(1,1)} \cdots p_{(m,1)}",\\
\vdots \\
"p_{(1,n)}  \cdots  p_{(m,n)}"\\
\end{array}\right]$$


$m$と$n$は等しいとは限らない。

例えば、花の絵を転置すると以下のようになる。

```python
['□□□□□□□□□■■■■□□□',
 '□□□□□□□□□■■■■□□□',
 '□□□□□□□□□■■■■■■□',
 '■■■■■■□□□■■■■■■□',
 '□■■■■■■□□□□■■■■□',
 '□□■■■■■□□□□■■■■■',
 '□■■■■■■■□□□□□□■■',
 '■■■■■■■■■■■■■■■■',
 '□■■■■■■■□□□□□□■■',
 '□□■■■■■□□□□■■■■■',
 '□■■■■■■□□□□■■■■□',
 '■■■■■■□□□■■■■■■□',
 '□□□□□□□□□■■■■■■□',
 '□□□□□□□□□■■■■□□□',
 '□□□□□□□□□■■■■□□□']
```

反時計回りに90度回転したようにみえるが、この絵が左右対象だったためにそう見えるだけで、行列の転置と同じ操作を行っている。

##### Step 1. 具体例を考え、テストを定義

In [18]:
def test_transpose():
    """
    transpose()をテストする。
    """
    picture = [
        '□■',
        '□■'
    ]
    picture_T = [
        '□□',
        '■■'
    ]
    assert transpose(picture) == picture_T
    # 以下必要に応じてテストケースを追加せよ(間違わない自信があれば追加しなくてもよい)

##### Step 2. `transpose()`を定義

In [16]:
def transpose(picture):
    """
    pictureを転置する
    """
    if not picture:
        return []

    rows = len(picture)
    cols = len(picture[0])

    transposed_picture = ['' for _ in range(cols)]

    for col in range(cols):
        for row in range(rows):
            transposed_picture[col] += picture[row][col]

    return transposed_picture

##### Step 3. テストの実行

In [19]:
# このセルは決して書き換えたり消したりしないこと
local_run(test_transpose)

### おまけ: エクセルファイルでの問題作成&ダウンロード

`get_sequence_lengths()`と`transpose()`が完成したら、以下を実行することでお絵かきロジックの問題をエクセルファイルとして生成し、問題のファイルをダウンロードできる。

In [20]:
# 必要ファイルのダウンロード
!wget --no-check-certificate -O picture_logic.7z "https://docs.google.com/uc?export=download&id=1J_hG-cXpwK5vk6qdMhhM9HsJ6B01-kAQ"
!7z x picture_logic.7z

--2024-06-28 10:52:21--  https://docs.google.com/uc?export=download&id=1J_hG-cXpwK5vk6qdMhhM9HsJ6B01-kAQ
Resolving docs.google.com (docs.google.com)... 74.125.142.113, 74.125.142.139, 74.125.142.100, ...
Connecting to docs.google.com (docs.google.com)|74.125.142.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1J_hG-cXpwK5vk6qdMhhM9HsJ6B01-kAQ&export=download [following]
--2024-06-28 10:52:21--  https://drive.usercontent.google.com/download?id=1J_hG-cXpwK5vk6qdMhhM9HsJ6B01-kAQ&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 108.177.98.132, 2607:f8b0:400e:c06::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|108.177.98.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 659 [application/octet-stream]
Saving to: ‘picture_logic.7z’

picture_logic.7z    100%[===================>]     659  --.-KB/s    in 0s     

In [21]:
# お絵かきロジックの完成絵
picture = [
 "□□□■□□□■□□□■□□□",
 "□□□■■□■■■□■■□□□",
 "□□□■■■■■■■■■□□□",
 "□□□■■■■■■■■■□□□",
 "□□□■■■■■■■■■□□□",
 "□□□■■■■■■■■■□□□",
 "□□□□■■■■■■■□□□□",
 "□□□□□□■■■□□□□□□",
 "□□□□□□□■□□□□□□□",
 "■■■■□□□■□□□■■■■",
 "■■■■□□□■□□□■■■■",
 "■■■■■■□■□■■■■■■",
 "■■■■■■□■□■■■■■■",
 "□□■■■■□■□■■■■□□",
 "□□■■■■■■■■■■■□□",
 "□□□□□■■■■■□□□□□"
]

# お絵かきロジックの問題ファイル
file_name = 'picture_logic.xlsx'

In [22]:
# このセルは決して書き換えたり消したりしないこと
import picture_logic
picture_logic.make_picture_logic_problem(picture, get_sequence_lengths, transpose, file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>